In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

--2025-10-20 18:23:13--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.007s  

2025-10-20 18:23:13 (10.6 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import KFold

In [6]:
df = pd.read_csv('course_lead_scoring.csv', sep = ',')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [8]:
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


In [9]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
categorical = ['lead_source', 'industry', 'employment_status', 'location']

In [10]:
df[categorical] = df[categorical].fillna('NA')
df[numerical] = df[numerical].fillna(0.0)

In [11]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 1)

In [12]:
df_train, df_val= train_test_split(df_full_train, test_size = 0.25, random_state = 1)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [13]:
for col in numerical:
  auc = roc_auc_score(y_train, df_train[col])
  if auc < 0.5:
    auc = 1-auc
  print(col, auc)

number_of_courses_viewed 0.7635680590007088
annual_income 0.5519578313253012
interaction_count 0.738270176293409
lead_score 0.6144993577250176


In [14]:
dv = DictVectorizer(sparse=False)
X_train_2=dv.fit_transform(df_train.to_dict(orient = 'records'))
X_val_2=dv.transform(df_val.to_dict(orient = 'records'))

In [15]:
model = LogisticRegression(solver='liblinear', C= 1.0, max_iter = 1000, random_state = 1)
model.fit(X_train_2, y_train)

LogisticRegression(max_iter=1000, random_state=1, solver='liblinear')

In [16]:
y_pred = model.predict_proba(X_val_2)[:, 1]
auc = roc_auc_score(y_val, y_pred)
round(auc,3)

np.float64(0.817)

In [36]:
thresholds = np.linspace(0, 1.0, 101)
for threshold in thresholds:
  y_pred_binary = (y_pred >= threshold).astype(int)
  tp = ((y_pred_binary == 1) & (y_val == 1)).sum()
  tn = ((y_pred_binary == 0) & (y_val == 0)).sum()
  fp = ((y_pred_binary == 1) & (y_val == 0)).sum()
  fn = ((y_pred_binary == 0) & (y_val == 1)).sum()
  precision = tp/(tp+fp) if (tp + fp) > 0 else 0.0
  recall = tp/(tp+fn)
  print(threshold, precision, recall)

0.0 0.0 nan
0.01 0.0 nan
0.02 0.0 nan
0.03 0.0 nan
0.04 0.0 nan
0.05 0.0 nan
0.06 0.0 nan
0.07 0.0 nan
0.08 0.0 nan
0.09 0.0 nan
0.1 0.0 nan
0.11 0.0 nan
0.12 0.0 nan
0.13 0.0 nan
0.14 0.0 nan
0.15 0.0 nan
0.16 0.0 nan
0.17 0.0 nan
0.18 0.0 nan
0.19 0.0 nan
0.2 0.0 nan
0.21 0.0 nan
0.22 0.0 nan
0.23 0.0 nan
0.24 0.0 nan
0.25 0.0 nan
0.26 0.0 nan
0.27 0.0 nan
0.28 0.0 nan
0.29 0.0 nan
0.3 0.0 nan
0.31 0.0 nan
0.32 0.0 nan
0.33 0.0 nan
0.34 0.0 nan
0.35000000000000003 0.0 nan
0.36 0.0 nan
0.37 0.0 nan
0.38 0.0 nan
0.39 0.0 nan
0.4 0.0 nan
0.41000000000000003 0.0 nan
0.42 0.0 nan
0.43 0.0 nan
0.44 0.0 nan
0.45 0.0 nan
0.46 0.0 nan
0.47000000000000003 0.0 nan
0.48 0.0 nan
0.49 0.0 nan
0.5 0.0 nan
0.51 0.0 nan
0.52 0.0 nan
0.53 0.0 nan
0.54 0.0 nan
0.55 0.0 nan
0.56 0.0 nan
0.5700000000000001 0.0 nan
0.58 0.0 nan
0.59 0.0 nan
0.6 0.0 nan
0.61 0.0 nan
0.62 0.0 nan
0.63 0.0 nan
0.64 0.0 nan
0.65 0.0 nan
0.66 0.0 nan
0.67 0.0 nan
0.68 0.0 nan
0.6900000000000001 0.0 nan
0.7000000000000001 0.0 n

/tmp/ipython-input-207769731.py:8: RuntimeWarning: invalid value encountered in scalar divide
  recall = tp/(tp+fn)


In [39]:
results_f1 = {}
for threshold in thresholds:
  y_pred_binary = (y_pred >= threshold).astype(int)
  tp = ((y_pred_binary == 1) & (y_val == 1)).sum()
  tn = ((y_pred_binary == 0) & (y_val == 0)).sum()
  fp = ((y_pred_binary == 1) & (y_val == 0)).sum()
  fn = ((y_pred_binary == 0) & (y_val == 1)).sum()
  precision = tp/(tp+fp)
  recall = tp/(tp+fn)
  F1 = 2 * precision * recall / (precision + recall)
  results_f1[threshold] = F1
  print(precision,recall)

0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6094420600858369 1.0
0.6120689655172413 1.0
0.6147186147186147 1.0
0.6173913043478261 1.0
0.6173913043478261 1.0
0.6200873362445415 1.0
0.6200873362445415 1.0
0.6200873362445415 1.0
0.6200873362445415 1.0
0.6200873362445415 1.0
0.6228070175438597 1.0
0.6228070175438597 1.0
0.6294642857142857 0.9929577464788732
0.6363636363636364 0.9859154929577465
0.6525821596244131 0.9788732394366197
0.6540284360189573 0.971830985915493
0.6540284360189573 0.971830985915493
0.6571428571428571 0.971830985915493
0.66666666666666

/tmp/ipython-input-3202047504.py:8: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)


In [19]:
max(results_f1, key =results_f1.get)

np.float64(0.5700000000000001)

In [20]:
cv = KFold(n_splits=5, shuffle=True, random_state=1)
model = LogisticRegression(solver='liblinear', C= 1.0, max_iter = 1000, random_state = 1)


In [25]:
def train(df_train, y_train, model):
    dicts = df_train.drop(['converted'],axis= 1).to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model.fit(X_train, y_train)

    return dv, model

In [26]:
def predict(df, dv, model):
    dicts = df.drop(['converted'],axis = 1).to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [27]:

from tqdm.auto import tqdm

In [31]:
n_splits = 5

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]
    y_train = df_train.converted.values
    y_val = df_val.converted.values

    dv, model = train(df_train, y_train, model)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

    print('%.3f +- %.3f' % (np.mean(scores), np.std(scores)))

0.806 +- 0.000
0.839 +- 0.033
0.818 +- 0.040
0.814 +- 0.035
0.822 +- 0.036


In [33]:
n_splits = 5

scores = []
for c in [0.000001, 0.001, 1]:
  kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
  model = LogisticRegression(solver
                             ='liblinear', C= c, max_iter = 1000, random_state = 1)
  for train_idx, val_idx in kfold.split(df_full_train):
      df_train = df_full_train.iloc[train_idx]
      df_val = df_full_train.iloc[val_idx]
      y_train = df_train.converted.values
      y_val = df_val.converted.values

      dv, model = train(df_train, y_train, model)
      y_pred = predict(df_val, dv, model)

      auc = roc_auc_score(y_val, y_pred)
      scores.append(auc)

      print('C=%s %.3f +- %.3f' % (c, np.mean(scores), np.std(scores)))

C=1e-06 0.557 +- 0.000
C=1e-06 0.538 +- 0.019
C=1e-06 0.555 +- 0.029
C=1e-06 0.556 +- 0.025
C=1e-06 0.560 +- 0.024
C=0.001 0.610 +- 0.114
C=0.001 0.651 +- 0.146
C=0.001 0.673 +- 0.148
C=0.001 0.693 +- 0.150
C=0.001 0.714 +- 0.156
C=1 0.722 +- 0.151
C=1 0.734 +- 0.150
C=1 0.738 +- 0.145
C=1 0.742 +- 0.140
C=1 0.750 +- 0.139
